In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 19 17:08:19 2024

@author: Rodrigo Meira
"""
from libs.eos_database import *
from libs.compressor_class import *
from libs.compression import *
from libs.gc_eos_soave import *
from libs.viscosity import *
from libs.plenum_system import *
from libs.simulation import *
from libs.model import *

from casadi import *
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import torch
from scipy.stats import qmc
from sklearn.model_selection import train_test_split

In [2]:
list_names = ["CH4", "C2H6", "C3H8", "iC4H10", "nC4H10", "iC5H12", "nC5H12", 
                  "nC6H14", "nC7H16", "nC8H18", "nC9H20", "nC10H22", "nC11H24", 
                   "nC12H26", "nC14H30", "N2", "H2O", "CO2", "C15+"]

nwe = [0.9834, 0.0061, 0.0015, 0.0003, 0.0003, 0.00055, 0.0004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003, 0.0, 0.0008, 0.0]

dict_composition= {list_names[i]: nwe[i] for i in range(len(nwe))}

mixture = Mixture(list_of_species, dict_composition)

volumn_desviation = [0] * 19

vlv = valve(kv=0.38)
Vpp = 2.0 
Lc = 2.0 
A1 = 2.6e-3
gas = gc_eos_class(mixture, 300, 4500, None, 1, 0, Aij, volumn_desviation, 'gas')
comp = CompressorClass()
visc = viscosity(mixture, volumn_desviation)
compressor = compression(gas, comp, visc)
plenum_sys = plenum(gas, compressor, vlv, Vpp, Lc, A1)

In [3]:
nAlphas = 300

sampler_N_RotS = qmc.LatinHypercube(d=1)  # d=1 porque estamos amostrando uma única variável
samples_N_RotS = sampler_N_RotS.random(n=nAlphas+1)
N_RotS = qmc.scale(samples_N_RotS, 600, 750).flatten()  # Redimensiona para 1D
sampler_alphas = qmc.LatinHypercube(d=1)
samples_alphas = sampler_alphas.random(n=nAlphas+1)
alphas = qmc.scale(samples_alphas, 0.36, 0.65).flatten()

nData = 40
dt = 2
timestep = 3
x0 = [14.9919, 339.69, 0.42885]
z0 = [6245.39, 6245.39, 321.672, 0.445562, 319.423, 0.503621, 320.097, 0.396345, 339.69, 0.42885, 0.514917]
u0 = [4500, 300, 600, alphas[0], 5000]

In [4]:
# sim = Simulation(plenum_sys, compressor, x0, z0, u0, nAlphas, alphas, N_RotS, nData, dt, timestep)
# x_values, z_values, time_steps, alpha_values, N_values, x_train, y_train, x_min, x_max, y_min,y_max = sim.run()

# sim_data = {
#     "x_values": x_values,
#     "z_values": z_values,
#     "time_steps": time_steps,
#     "x_train": x_train,
#     "y_train": y_train,
#     "x_min": x_min,
#     "x_max": x_max,
#     "y_min": y_min,
#     "y_max": y_max

# }

# np.savez(
#     "simulation_data.npz",
#     x_values=x_values,
#     z_values=z_values,
#     time_steps=time_steps,
#     alpha_values=alpha_values,
#     N_values=N_values,
#     x_train=x_train,
#     y_train=y_train,
#     x_min=x_min,
#     x_max=x_max,
#     y_min = y_min,
#     y_max = y_max

# )


In [5]:
data = np.load("simulation_data.npz")

x_values = data["x_values"]
z_values = data["z_values"]
time_steps = data["time_steps"]
alpha_values = data["alpha_values"]
N_values = data["N_values"]
x_train = data["x_train"]
y_train = data["y_train"]
x_min = data["x_min"]
x_max = data["x_max"]
y_min = data["y_min"]
y_max = data["y_max"]


x_min = torch.tensor(x_min, dtype = torch.float32)
x_max = torch.tensor(x_max, dtype = torch.float32)
y_min = torch.tensor(y_min, dtype = torch.float32)
y_max = torch.tensor(y_max, dtype = torch.float32)
x_train = torch.tensor(np.array(x_train), dtype=torch.float32)
y_train = torch.tensor(np.array(y_train), dtype=torch.float32)
x_train_split, X_val, y_train_split, y_val = train_test_split(
    x_train, 
    y_train, 
    test_size=0.3,  
    random_state=42,  
    shuffle=True      
)
y_train_split = y_train_split.squeeze()
y_val = y_val.squeeze()
train_dataset = torch.utils.data.TensorDataset(x_train_split, y_train_split)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size= 64, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size= 64, shuffle=False)

In [6]:
model = MyModel(128, dt, x_max, x_min, y_min, y_max, plenum_sys)# Criar a instância do modelo novamente

In [7]:
# modelo = MyModel(200, dt, x_max, x_min, y_min, y_max, plenum_sys)  # mesma definição usada antes
# modelo.load_state_dict(torch.load('pesos_modelo.pth'))


In [9]:
train_loss_values = model.train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    lr= 1e-3,
    epochs=70,
    optimizers=torch.optim.Adam,
    patience=5,
    factor=0.5,
    gas = gas
)

Batch 0 completed in 6.79s | Data: 13578.3613 | Phys: 18.2246 | Prof: 427.6821
Batch 1 completed in 3.48s | Data: 17890.0586 | Phys: 16.3881 | Prof: 448.2281
Batch 2 completed in 3.49s | Data: 16555.8359 | Phys: 17.3265 | Prof: 331.9055
Batch 3 completed in 3.61s | Data: 11879.9453 | Phys: 17.7669 | Prof: 211.9219
Batch 4 completed in 3.47s | Data: 17112.6562 | Phys: 18.7026 | Prof: 222.3085
Batch 5 completed in 3.48s | Data: 11224.7832 | Phys: 15.2649 | Prof: 122.0032
Batch 6 completed in 3.60s | Data: 14973.6680 | Phys: 15.1641 | Prof: 97.7177
Batch 7 completed in 0.53s | Data: 11807.1777 | Phys: 15.2735 | Prof: 91.3122
Batch 8 completed in 3.49s | Data: 11113.0713 | Phys: 14.2181 | Prof: 56.4868
Batch 9 completed in 3.50s | Data: 12984.5742 | Phys: 13.6822 | Prof: 51.6035
Batch 10 completed in 3.57s | Data: 8299.0059 | Phys: 12.9153 | Prof: 75.6053
Batch 11 completed in 3.49s | Data: 8069.6328 | Phys: 11.7932 | Prof: 49.3627
Batch 12 completed in 3.51s | Data: 9116.6426 | Phys: 12.1

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'model.pth')


In [ ]:
nAlphas_teste = 7

sampler_N_RotS_teste = qmc.LatinHypercube(d=1)  # d=1 porque estamos amostrando uma única variável
samples_N_RotS_teste = sampler_N_RotS_teste.random(n=nAlphas_teste+1)
N_RotS_teste = qmc.scale(samples_N_RotS_teste, 600, 750).flatten()  # Redimensiona para 1D
sampler_alphas_teste = qmc.LatinHypercube(d=1)
samples_alphas_teste = sampler_alphas_teste.random(n=nAlphas_teste+1)
alphas_teste = qmc.scale(samples_alphas_teste, 0.35, 0.65).flatten()  

sim2 = sim = Simulation(plenum_sys, compressor, x0, z0, u0, nAlphas_teste, alphas_teste, N_RotS_teste, 40, dt, timestep)
x_values_teste, z_values_teste, time_steps, alpha_values_teste, N_values_teste, x_teste, y_teste, x_min, x_max, y_min, y_max = sim.run()
x_teste_np = x_teste.reshape(1,277, 3, 7)
interval3 = np.linspace(0, 40*nAlphas_teste, len(x_teste))

In [ ]:
import torch

# Rodar a inferência corretamente
model.eval()  # Importante colocar a rede em modo de avaliação
with torch.no_grad():
    y_pred = model(x_teste)

# Checar se y_pred realmente varia
for name, param in model.named_parameters():
    print(f"{name}: mean={param.mean().item()}, std={param.std().item()}")



In [ ]:
torch.onnx.export(
    model,
    x_teste,
    "modelo_simples6.onnx",
    export_params=True,
    opset_version=17,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch_size"},  # Permite batch variável
        "output": {0: "batch_size"},
    },
)

In [ ]:
import onnxruntime as ort
import numpy as np

# Converter o tensor para NumPy e ajustar o dtype
x_teste_np = x_teste.detach().cpu().numpy().astype(np.float32)  # Garanta float32

# Se o modelo esperar um batch dimension (ex: [batch, seq_len, features]), adicione:
if x_teste_np.ndim == 1:
    x_teste_np = x_teste_np[np.newaxis, ...]  # Adiciona dimensão de batch (batch=1)

# Carregar o modelo ONNX
ort_session = ort.InferenceSession("modelo_simples6.onnx")

# Executar inferência corretamente
input_name = ort_session.get_inputs()[0].name
outputs = ort_session.run(None, {input_name: x_teste_np})[0]

In [ ]:
import matplotlib.pyplot as plt

# Remover dimensões extras de y_pred e y_teste (se houver)
outputs = outputs.squeeze()
y_teste = y_teste.squeeze()

# Número de variáveis de saída
num_outputs = y_teste.shape[1]

# Ajustar o tamanho da lista time_steps para corresponder aos dados de teste
time_steps = time_steps[:y_teste.shape[0]]
 
# Criar os gráficos separadamente para cada saída
for i in range(num_outputs):
    plt.figure(figsize=(8, 4))  # Criar uma nova figura para cada gráfico
    plt.plot(time_steps, y_teste[:, i], label="Saída Esperada (y_teste)", color="red", linestyle="--")
    plt.plot(time_steps, outputs[:, i], label="Saída da Rede (outputs)", color="blue", linestyle="-")
    
    plt.xlabel("Time Step")
    plt.ylabel("Valor")
    plt.title(f"Saída {i}")  # Título do gráfico indicando o índice da saída
    plt.legend()
    plt.grid(True)
    
    plt.show()  # Mostrar o gráfico

"[0, 1, 2, 3, 5, 7, 9, 11]"

In [ ]:
print(x_teste.shape)

In [ ]:
onnx_model_path = "./modelo_simples6.onnx"
onnx_session = ort.InferenceSession(onnx_model_path)

massFlowrate_pred = [x_teste[0, 0, 0].item(), x_teste[0, 1, 0].item(), x_teste[0, 2, 0].item()]
Temperatura = [x_teste[0, 0, 1].item(), x_teste[0, 1, 1].item(), x_teste[0, 2, 1].item()]
PlenumPressure_pred = [x_teste[0, 0, 2].item(), x_teste[0, 1, 2].item(), x_teste[0, 2, 2].item()]
Pressure2 = [x_teste[0, 0, 3].item(), x_teste[0, 1, 3].item(), x_teste[0, 2, 3].item()]
Temperatura2 = [x_teste[0, 0, 4].item(), x_teste[0, 1, 4].item(), x_teste[0, 2, 4].item()]

input_tensor = torch.zeros((1, 3, 7), dtype=torch.float32) 

tm1 = time.time()

for i in range(len(interval3)):
    # Atualizar os valores retroalimentados (massFlowrate e PlenumPressure)
    input_tensor[0, :, 0] = torch.tensor(massFlowrate_pred[-3:])
    input_tensor[0, :, 1] = torch.tensor(Temperatura[-3:])
    input_tensor[0, :, 2] = torch.tensor(PlenumPressure_pred[-3:])
    input_tensor[0, :, 3] = torch.tensor(Pressure2[-3:])
    input_tensor[0, :, 4] = torch.tensor(Temperatura2[-3:])
    input_tensor[0, :, 5] = x_teste[i, :, 5]
    input_tensor[0, :, 6] = x_teste[i, :, 6]
    # Copiar diretamente as variáveis restantes da entrada (colunas 2 a 6)

    # Previsão com ONNX
    onnx_inputs = {'input': input_tensor.numpy()}
    onnx_outputs = onnx_session.run(None, onnx_inputs)

    # Adicionar as novas previsões
    massFlowrate_pred.append(onnx_outputs[0][0][0])
    Temperatura.append(onnx_outputs[0][0][1])
    PlenumPressure_pred.append(onnx_outputs[0][0][3])
    Pressure2.append(onnx_outputs[0][0][4])
    Temperatura2.append(onnx_outputs[0][0][11])
tm2 = time.time()
timeteste = tm2 - tm1

print(f"Tempo de teste: {timeteste:.4f}s")
Temperatura = np.array(Temperatura)
lista = [massFlowrate_pred, Temperatura, PlenumPressure_pred, Pressure2, Temperatura2]
lista = np.array(lista)
print(lista.shape)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Converter listas para numpy arrays
massFlowrate_pred = np.array(massFlowrate_pred[3:])
Temperatura_pred = np.array(Temperatura[3:])
PlenumPressure_pred = np.array(PlenumPressure_pred[3:])
Pressure2_pred = np.array(Pressure2[3:])
Temperatura2_pred = np.array(Temperatura2[3:])

# Garantir que y_teste está no formato de numpy array
y_teste = np.array(y_teste)

# Plotar os gráficos
plt.figure(figsize=(15, 12))

# 1. Mass Flowrate
plt.subplot(3, 2, 1)
plt.plot(massFlowrate_pred, label='Predito')
plt.plot(y_teste[:, 0], label='Real', linestyle='--')
plt.title('Mass Flowrate')
plt.legend()

# 2. Temperatura
plt.subplot(3, 2, 2)
plt.plot(Temperatura_pred, label='Predito')
plt.plot(y_teste[:, 1], label='Real', linestyle='--')
plt.title('Temperatura')
plt.legend()

# 3. Plenum Pressure
plt.subplot(3, 2, 3)
plt.plot(PlenumPressure_pred, label='Predito')
plt.plot(y_teste[:, 3], label='Real', linestyle='--')  # Índice 4: conferir se bate com seu output
plt.title('Plenum Pressure')
plt.legend()

# 4. Pressure 2
plt.subplot(3, 2, 4)
plt.plot(Pressure2_pred, label='Predito')
plt.plot(y_teste[:, 4], label='Real', linestyle='--')  # Índice 5: conferir
plt.title('Pressure 2')
plt.legend()

# 5. Temperatura 2
plt.subplot(3, 2, 5)
plt.plot(Temperatura2_pred, label='Predito')
plt.plot(y_teste[:, 11], label='Real', linestyle='--')  # Índice 11: conferir
plt.title('Temperatura 2')
plt.legend()

plt.tight_layout()
plt.show()
